In [3]:
import pandas as pd 
from PIL import Image 
import cv2
import numpy as np
import os
import colorsys
import tqdm

# Get main colors

In [131]:
MAIN_COLORS_HSL = {
    'pink': (320, 340),
    'red': ((340, 360), (0, 15)), 'orange': (15, 45),
    'yellow': (45, 55), 'green': (55, 165),
    'blue': (165, 240),
    'violet': (240, 320)
    }

def find_neighbour_color(color_hsv, bl, wh):
    ans = ''
    if bl <= 15 and wh <= 15:
        return 'dark'
    for i, j in MAIN_COLORS_HSL.items():
        if i == 'red':
            if j[0][0] <= color_hsv <= j[0][1] or j[1][0] <= color_hsv <= j[1][1]:
                ans = 'red'
                break
        elif j[0] <= color_hsv <= j[1]:
            ans = i
            break
    if ans == 'red' and ((bl <= 28 and wh >= 78) or wh >= 90):
        ans = 'pink'
    elif ans == 'orange' and  (bl <= 28 and wh >= 78 or wh >= 90):
        ans = 'peach'
    elif (ans == 'orange' or ans == 'red') and (wh <= 25 or (wh <= 45 and bl <= 45)):
        ans = 'brown'
    return ans

In [ ]:
data = pd.read_csv('data.csv', delimiter=';')
data['bl'] = data['bl'] * 100
data['wh'] = data['wh'] / 255 * 100
data['color'] = data.apply(lambda x: find_neighbour_color(x.color_hsv, x.bl, x.wh), axis=1)
data

In [147]:
data.groupby('color').img_path.count()

color
blue      1247
brown      798
dark       492
green      737
orange     986
peach     1314
pink       797
red        575
violet     412
yellow     188
Name: img_path, dtype: int64

In [153]:
for i in data[data.color == 'peach'].img_path[200:400]:
    im = Image.open(i) 
    im.show()


In [155]:
data.to_csv('data_with_colors.csv', index=False)

# Clothes type

In [1]:
import os

In [ ]:
python3 train.py --epochs 100 --workers 4 --batch-size 32 --data data/clothes.yaml --img 640 640 --cfg cfg/training/yolov7_clothes-tiny.yaml --weights 'yolov7-tiny.pt' --name yolov7_tiny_clothes_fixed_res --hyp data/hyp.scratch.tiny.yaml

In [4]:
data = pd.read_csv('data_with_colors.csv')
data.img_path = data.img_path.apply(lambda x: x.split('/')[1])
data

,img_path,color_hsv,bl,wh,color
0,image6561.png,40.000000,1.209677,97.254902,peach
1,image10104.png,28.085106,20.524017,89.803922,peach
2,image9652.png,215.555556,27.551020,38.431373,blue
3,image3419.png,290.000000,3.260870,72.156863,violet
4,image5068.png,172.500000,19.047619,16.470588,blue
...,...,...,...,...,...
7541,image3400.png,141.818182,23.913043,18.039216,green
7542,image8555.png,16.000000,64.516129,36.470588,orange
7543,image9893.png,169.565217,19.327731,46.666667,blue
7544,image1217.png,354.000000,89.285714,21.960784,brown


In [5]:
classes = list(map(str.strip, open('classes.txt').readlines()))

In [6]:
images = next(os.walk('images'))[2]
data = data[data.img_path.isin(list(map(lambda x: x.split('-')[-1], images)))]

In [8]:
labels = next(os.walk('labels'))[2]
a = []
for i in labels:
    clth = list(map(int, open('labels/'+i).read().replace('\n', ' ').split()[::5]))
    a.append((i.replace('txt', 'png'), clth))
labels_data = pd.DataFrame(a)
labels_data['img_path'] = labels_data[0].apply(lambda x: x.split('-')[-1])


In [9]:
data = labels_data.set_index('img_path').join(
    data.set_index('img_path')).reset_index().drop(
        columns='img_path').rename(columns={0: 'img_path', 1: 'clothes'})
data['clothes'] = data['clothes'].apply(lambda x: ','.join(map(str, set(x))))
data.to_csv('marked_images.csv', index=False, sep=';')

In [59]:
img = 'image10044.png'
data[data.img_path == img]

,img_path,clothes,color_hsv,bl,wh,color
250,image10044.png,"[3, 8, 13, 15]",6.666667,7.114625,99.215686,pink


In [60]:
all_colors = ['red', 'pink', 'peach', 'orange', 'yellow', 'green', 'blue', 'violet', 'dark', 'brown']

In [69]:
# def find_similar_img(img):
new_data = data.copy()
new_data['color_id'] = new_data.color.apply(lambda x: all_colors.index(x))
img_ = new_data[new_data.img_path == img]
new_data['color_diff'] = new_data.color_id.apply(lambda x: (10 - abs(x - img_.color_id)) / 10  )
new_data
new_data['similar_clothes'] = new_data.clothes.apply(lambda x: sum([1 for i in x if (i in img_.clothes.to_list()[0])]))
new_data['similar_clothes'] /= new_data['similar_clothes'].max()
new_data['diff'] = new_data['color_diff'] / 2 * 0 +  new_data['similar_clothes']
new_data.sort_values('diff', ascending=False).head(10)

,img_path,clothes,color_hsv,bl,wh,color,color_id,color_diff,similar_clothes,diff
262,image10124.png,"[0, 0, 0, 0, 13, 15, 0, 8, 3]",55.384615,6.632653,76.862745,green,5,0.6,1.0,1.0
425,image10024.png,"[19, 8, 8, 9, 15, 13, 0]",30.000000,23.157895,37.254902,brown,9,0.2,1.0,1.0
110,image10163.png,"[8, 3, 15, 13, 0, 9]",21.818182,23.655914,36.470588,brown,9,0.2,1.0,1.0
330,image1016.png,"[8, 3, 15, 0, 13]",204.444444,44.262295,71.764706,blue,6,0.5,1.0,1.0
241,image10539.png,"[8, 19, 13, 0, 0, 15, 15]",27.600000,26.041667,75.294118,orange,3,0.8,1.0,1.0
499,image10544.png,"[19, 13, 15, 0, 0, 13, 8]",19.655172,26.728111,85.098039,peach,2,0.9,1.0,1.0
122,bdc42900-image76.png,"[15, 13, 3, 8]",240.000000,2.390438,98.431373,blue,6,0.5,1.0,1.0
440,image10150.png,"[3, 8, 0, 13, 15]",0.000000,0.416667,94.117647,pink,1,1.0,1.0,1.0
422,image10595.png,"[13, 8, 3, 4, 15, 0]",0.000000,0.000000,95.686275,pink,1,1.0,1.0,1.0
250,image10044.png,"[3, 8, 13, 15]",6.666667,7.114625,99.215686,pink,1,1.0,1.0,1.0


In [10]:
data

,img_path,clothes,color_hsv,bl,wh,color
0,image10760.png,"0,2,6,13,15,19",36.000000,2.024291,96.862745,peach
1,image10006.png,"0,1,6,13,20",60.000000,0.408163,96.078431,green
2,image10589.png,"1,3,6",27.272727,4.526749,95.294118,peach
3,8c731166-image28.png,"20,6,15",25.090909,56.701031,38.039216,orange
4,66084130-image8.png,"0,10,13,15",45.000000,1.581028,99.215686,peach
...,...,...,...,...,...,...
616,image10779.png,"0,10,13,15",353.333333,70.183486,85.490196,red
617,image10023.png,"10,13,15",204.878049,19.902913,80.784314,blue
618,image10745.png,"0,4,11,15,18",30.000000,40.816327,19.215686,brown
619,image10751.png,"0,1,11,13,20",355.714286,81.666667,47.058824,red
